In [ ]:
class clustersegmentation:

    def __init__(self,input_image_path): # __init__() is built-in function

        self.input_image_path   = input_image_path
            #self.seed               = seed  # argument can be passed through class or through the function
            #self.number_of_cluster  = number_of_cluster

    def otsusegmentation(self):
            """
            find the optimal thresholding, 
            """
        image                 = imread(self.input_image_path)
        otsu                  = threshold_otsu(image)
        binary                = image > otsu

        return binary    
    def LevelSetActiveContour(self):
            """
            algorithm: 
            1.set initial level set using checkboard formular, 3.14/5 to minimize original shape, we can also use original
              shape as initial level set
            2.

            compute the energy difference until it converges
            , new level set, enery is difined as sum of pixel value of internal erngy and external energy

            https://scikit-image.org/docs//auto_examples/segmentation/plot_chan_vese.html
            https://github.com/scikit-image/scikit-image/blob/main/skimage/segmentation/_chan_vese.py
            used for initial contour of object is not clear;
            euclidean distance transform for binary image is the distance from pixel, the further assigns higher value
            it requires binary image or segmented image with background value is 0
            level_set = inspect.getsource(chan_vese)
            print(level_set)
            init_level_set="checkerboard"

            mu high value produce round image, small mu detects small object
            lambda1 computes the output image true value; lambda1 < than lambda 2, true image has higher value 
            lambda2 compute output image false value;typically are between 0- 1. greater than one, for irregular shape contour
            tol(tolerance), level set toleranace between each iteration, monitor convergence,
            smaller value can see better result
            dt multiplication factor to speed up computation, 
            checkboard is starting level set, has fast convergence,initial_level_set can be same as shape of oroginal image
            disk the starting level set
            small disk , another starting level set
            entended output, boolean value, if false only segented image array is produce
            max_iter used to monitor whether it converges or not

            """
        image  = imread(self.input_image_path)
        image  = img_as_float(image)
        cv     = chan_vese(image, mu=0.2, lambda1=20, lambda2=20, tol=1e-12, max_iter=200,dt=0.5,init_level_set="checkerboard",  extended_output=True)


        return cv[0] #at certain iteration seems final segmented image binary
            #return cv[1] #final image not binary
            #return cv[2] #energy over number of iteration

    def watershedsegmentation(self):
            """
            https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_watershed.html#:~:text=The%20watershed%20is%20a%20classical,a%20local%20topography%20(elevation).
            https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.distance_transform_edt.html#scipy.ndimage.distance_transform_edt
            https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_watershed/py_watershed.html 
            input_image_path    = 'C:\\Users\\Kevin\\Desktop\\image_processing\\fc_images_bmp\\Jurkats\\5_Ch1.ome.bmp' 
            input_image_path    = 'C:\\Users\\Kevin\\Desktop\\image_processing\\otsusegmentation\\Jurkats\\7587_Ch1.ome_otsu.bmp' 
            input_image_path   = 'C:\\Users\\Kevin\\Desktop\\image_processing\\fc_images_bmp\\Bone Marrow\\7_Ch1.ome.bmp' 
            image = watershedsegmentation(input_image_path)
            plt.imshow(image,cmap="gray")
            """
        image       = imread(self.input_image_path)
        otsu        = threshold_otsu(image)
        binary      = image > otsu          #global threshold
        distance    = ndimage.distance_transform_edt(binary) 
                                                # calculate euclidean distance, same shape as input array 
          #  coords      = peak_local_max(distance, footprint=np.ones((3, 3)), labels=binary) #finds peak of image as coordinate
        coords      = peak_local_max(distance) 
                                                # return coordinates of local maximum pixels
        mask        = np.zeros(distance.shape, dtype=bool) 
                                                # create a zero 2D array
            #markers, _ = ndimage.label(mask)   # markers is labeled value, _ returns number of features/ found objects,
                                                # is just a variable
        mask[tuple(coords.T)] = True        # not neccessary to use tuple, convert all of rows into mentioned value
        markers, _ = ndimage.label(mask)
                                                # return two values, markers is labeled value, _ returns number of features/ found objects,
                                                # is just a variable
        labels = watershed(-distance, markers, mask=binary)
                                                #  - sign menas turn black pixel into white and vice versa 
        return labels

    def watershedopencv(self):
            """
            https://www.youtube.com/watch?v=M1mJsJ5M4iE


            """
        img = cv.imread(self.input_image_path)  # result is 3 channel
        image = img[:,:,0]                      # get rid of channel
        pixels_to_um = 0.454                    # 
        ret1, thresh = cv.threshold(image, 2,255,cv.THRESH_BINARY + cv.THRESH_OTSU) 
                                                    # obtain the edge of image using OTSU thresholding
        kernel = np.ones((3,3), np.uint8)       # go through each pixel perform element-wise multiplication using 3x3 kernel
        opening = cv.morphologyEx(thresh,cv.MORPH_OPEN, kernel, iterations = 0 ) 
                                                    # basically expand the edges of binary image
                                                    # cv.MOPH_OPEN means expand  
                                                    # iteration means number of times it applied
        opening_clear = clear_border(opening)   # neibouring/border of labeling pixel become 0
                                                    # background pixel also becomes 0
        sure_bg       = cv.dilate(opening_clear, kernel, iterations = 1) 
                                                    # make background image expand after clear border, contour image
                                                    # at this step we found background including unknown from dilate step
                                                    # basically enlarge the binary image of background, the reason to do this 
                                                    # is because in boundary area not sure whether it is foreground or background  
        dist_transform = cv.distanceTransform(opening_clear, cv.DIST_L2,5) 
                                                    # or we can use erosion,
                                                    # print(dist_transform.max()) #
        ret2,sure_fg = cv.threshold(dist_transform, 0.5*dist_transform.max(),255,0) 
                                                    # make foreground larger 
                                                    # only show 50% of distance, better erosion as
                                                    # erosion might lose small pixel
        sure_fg         = np.uint8(sure_fg)                 
        unknown         = cv.subtract(sure_bg,sure_fg)      
                                                    # result is the edge around the centre which is unknown
        ret3,markers    = cv.connectedComponents(sure_fg)   
                                                    # find the pixels that connected to it, either 4 or 8 connected 


        markers         = markers +1            # convert backgronud to 10 instead of 0; 
        markers[unknown ==255] = 0              # mark unknown region to 0
        markers = cv.watershed(img,markers)     # img has to be 3 channel image not gray scale; 
                                                    # -1 means boundary  
                                                    # watershed treat pixel 0 as uknown region, 
        return markers        


    def k_mean_segmentation(self,a):
            '''
            drawback is stucking at local minimum. The result is accetable base on industrial report. 
            input value is an image and number of clustering
            This algorithm takes initial k seed point from dataset. It can be any seed point, it does not matter that much as it
            will converge eventually just matter of time. Use that seed point, calculate eucleanian distance and form each pixel;
            second step is take mean value of that cluster and make it become new seed point; 
            base on the new seed point, it repeats the iteration and cluster;
            it keeps iterating until no more new cluster has formed which means solution has been converged; 
            image format needs to be int8 values range from 0 to 255, otherwise, needs to transform
            '''
            image           = imread(self.input_image_path)
            attempts        = 10
            criteria        = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0) 
            #10 is number of iteration and epsilon value is 1, whichevery is small
            image_pixel_values = image.reshape(-1,1) #convert to -1x1 matrix as it will plot randomly
            image_1_convert =  np.float32(image_pixel_values) #convert pixel value to floating 
            ret,labels,centers = cv.kmeans(image_1_convert,a,None,criteria,attempts,cv.KMEANS_RANDOM_CENTERS)
            # cv.KMEANS_RANDOM_CENTERS means randomly asssign a seed (y,x)
            #pixel position is written in (y,x)
            #image shape is writen in height and width which is also y, and x
            # ret is compactness means sum of square from centroid
            # centers is center of each cluster
            # labels is number of cluster lable if k is 3, lables are 0, 1, 2,
        centers     = np.uint8(centers) #convert to integer 8
        labels      = labels.flatten()
        segmented   = centers[np.reshape(labels,image.shape)]

        number_of_cluster_0 = (segmented == centers[0])
        number_of_cluster_1 = (segmented == centers[1])
        number_of_cluster_2 = (segmented == centers[2])

        a = [number_of_cluster_0.sum(),number_of_cluster_1.sum(),number_of_cluster_2.sum()]
        b = a.index(max(a))
        if b == 0:
            final_cluster = np.where(segmented == centers[0],np.float(0),255)
        elif b==1:
            final_cluster = np.where(segmented == centers[1],np.float(0),255)
        elif b == 2:
            final_cluster = np.where(segmented == centers[2],np.float(0),255)


            #if number_of_cluster_0.sum() < number_of_cluster_1.sum():

             #   final_cluster = np.where(segmented == centers[1],np.float(0),255)
    #         else:
    #             final_cluster = np.where(segmented == centers[0],np.float(0),255)

    #         number_of_cluster_0   = np.where(segmented == centers[0],np.float(0),255) #replacement number has to be float as well
    #         number_of_cluster_1   = np.where(segmented == centers[1],np.float(0),255)
    #         number_of_cluster_2   = np.where(segmented == centers[2],np.float(0),255)
            #number_of_cluster_3   = np.where(segmented == centers[3],np.float(0),255)


        #     if number_of_cluster_1 < num

    #         return number_of_cluster_0.reshape(image.shape[0],image.shape[1])
    #         return number_of_cluster_1.reshape(image.shape[0],image.shape[1])
    #         return number_of_cluster_2.reshape(image.shape[0],image.shape[1])
    #         return number_of_cluster_3.reshape(image.shape[0],image.shape[1])
    #         return number_of_cluster_4.reshape(image.shape[0],image.shape[1])
        return final_cluster.reshape(image.shape[0],image.shape[1])

    def region_growing(self,seed):
            """
            https://github.com/Panchamy/RegionGrowing/blob/master/RegionGrowing.py

            """
            #Parameters for region growing
        img=imread(self.input_image_path)
        seed =seed
        neighbors = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        region_threshold = 0.2
        region_size = 1
        intensity_difference = 0
        neighbor_points_list = []
        neighbor_intensity_list = []

            #Mean of the segmented region
        region_mean = img[seed]

            #Input image parameters
        height, width = img.shape
        image_size = height * width

            #Initialize segmented output image
        segmented_img = np.zeros((height, width, 1), np.uint8)

            #Region growing until intensity difference becomes greater than certain threshold
        while (intensity_difference < region_threshold) & (region_size < image_size):
                #Loop through neighbor pixels
            for i in range(4):
                    #Compute the neighbor pixel position
                x_new = seed[0] + neighbors[i][0]
                y_new = seed[1] + neighbors[i][1]

                    #Boundary Condition - check if the coordinates are inside the image
                check_inside = (x_new >= 0) & (y_new >= 0) & (x_new < height) & (y_new < width)

                    #Add neighbor if inside and not already in segmented_img
                if check_inside:
                    if segmented_img[x_new, y_new] == 0:
                        neighbor_points_list.append([x_new, y_new])
                        neighbor_intensity_list.append(img[x_new, y_new])
                        segmented_img[x_new, y_new] = 255

                #Add pixel with intensity nearest to the mean to the region
                distance = abs(neighbor_intensity_list-region_mean)
                pixel_distance = min(distance)
                index = np.where(distance == pixel_distance)[0][0]
                segmented_img[seed[0], seed[1]] = 255
                region_size += 1

                #New region mean
                region_mean = (region_mean*region_size + neighbor_intensity_list[index])/(region_size+1)

                #Update the seed value
                seed = neighbor_points_list[index]
                #Remove the value from the neighborhood lists
                neighbor_intensity_list[index] = neighbor_intensity_list[-1]
                neighbor_points_list[index] = neighbor_points_list[-1]

            return segmented_img

        def HistEqualization(self):
            """
            regular histogram only computes number of pixel, histogram equalization is used cumulative
            sum to add previous histogram to neibouring one. so the pixel value gets larger, it 
            basically adds previous value into it. Purpose is to increase contrast
            a= plt.hist(image.flatten(),bins = 256,histtype='step')
            """
            image = imread(self.input_image_path)
            imgae = rgb2gray(image)

            #a                  = plt.hist(image.flatten(),bins = 256,histtype='step') #obtain histogram of original image
            image_histequal    = exposure.equalize_hist(image)
            #img_cdf, bins      = exposure.cumulative_distribution(image, 256)    
            #p2, p98            = np.percentile(image, (7, 93)) # set image intensity between 2 to 98 percentage??
            #img_rescale        = exposure.rescale_intensity(image, in_range=(p2, p98))
            #img_adapteq        = exposure.equalize_adapthist(image, clip_limit=0.1) #showing more contrast for local pixel

            return image_histequal

        def Hist(self):
            """
            regular histogram only computes number of pixel, histogram equalization is used cumulative
            sum to add previous histogram to neibouring one. so the pixel value gets larger, it 
            basically adds previous value into it. Purpose is to increase contrast
            a= plt.hist(image.flatten(),bins = 256,histtype='step')
            """
            image = imread(self.input_image_path)
            hist_image = plt.hist(image.flatten(),bins = 256,histtype='step')
            return hist_image

        def canny_dilated_image(self):
            """
            canny algorithm is used to detect edge of image, dilate function is used blur edge which can help 
            binary image generation

            """

            image = imread(self.input_image_path)
            canny_image = cv.Canny(image, 0,255)
            dilated_canny_image = cv.dilate(canny_image,None)

            return dilated_canny_image

        def binary_mask(input_image_path):
            original           = imread(input_image_path)
            image              = cv.dilate(original,None) # help find the edges
            cnt = sorted(cv.findContours(image, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)[-2], key=cv.contourArea)[-1] 
            #[-1] provides largest contour as it is sorted; 
            masked = []
            segmented = []
            mask = np.zeros((original.shape[0],original.shape[1]), np.uint8)
            masked.append(cv.drawContours(mask, [cnt],-1, 255, -1))
            dst = cv.bitwise_and(original, original, mask=mask)
            segmented.append(dst)
            return masked[0]


        def binary_mask_and_original(input_image_path):
            original           = imread(input_image_path)
            image              = cv.dilate(original,None) # help find the edges
            cnt = sorted(cv.findContours(image, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)[-2], key=cv.contourArea)[-1] 
            #[-1] provides largest contour as it is sorted; 
            masked = []
            segmented = []
            mask = np.zeros((original.shape[0],original.shape[1]), np.uint8)
            masked.append(cv.drawContours(mask, [cnt],-1, 255, -1))
            dst = cv.bitwise_and(original, original, mask=mask)
            segmented.append(dst)
            return original[0]
